In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import numpy as np
import torch
from PIL import Image

from DataPreprocess.getBBox import get_BBox
from DataPreprocess.detect3DMM import detect_3dmm

root_path = '/data1/wc_log/zxy/custom_dataset/jjk_video_1'
save_path = '/data1/wc_log/zxy/custom_dataset/jjk_video_1/train_1'

In [2]:
from modules.ConditionBranch import Condition_Branch
from face_alignment.detection.sfd.sfd_detector import SFDDetector

# get face detector
face_detector = SFDDetector(device='cuda')

# get condition_Branch
condition_branch = Condition_Branch()

/home/wenchi/miniconda3/envs/diffusion/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/wenchi/miniconda3/envs/diffusion/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


creating the FLAME Decoder
trained model found. load /home/wenchi/zxy/HSD/ControlNet/utils/DECA/data/deca_model.tar


/home/wenchi/miniconda3/envs/diffusion/lib/python3.9/site-packages/pytorch3d/io/obj_io.py:548: UserWarning: Mtl file does not exist: /home/wenchi/zxy/HSD/ControlNet/utils/DECA/data/template.mtl
  warnings.warn(f"Mtl file does not exist: {f}")


In [4]:
# get all images from root_path dir
image_names = os.listdir(root_path)
image_names = sorted(image_names)
image_paths = [os.path.join(root_path, image) for image in image_names]

image_nums = len(image_paths)

# make dir path
if root_path != save_path :
    os.makedirs(save_path, exist_ok=True)

imgs_numpy_list = [np.asarray(Image.open(imagepath).convert("RGB").resize((512, 512))) for imagepath in image_paths]
imgs_numpy = np.asarray(imgs_numpy_list).transpose(0, 3, 1, 2) # (all, 3, H, W)
imgs_tensor = torch.from_numpy(imgs_numpy)
batch_size = image_nums

# get bbox
bboxlist = get_BBox(imgs_tensor, face_detector, batch_size=batch_size) # (2, 4)

# get 3DMM dict
dict_3DMM = detect_3dmm(bboxlist, imgs_numpy, condition_branch.deca, save_folder_path = save_path, batch_size = batch_size) # source, target

# save images
if root_path != save_path :
    for i in range(image_nums):
        image = Image.fromarray(imgs_numpy_list[i])
        image.save(os.path.join(save_path, '{}.png'.format(str(i).zfill(8))))